In [1]:
import geopandas as gpd

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov='51'

In [3]:
data_gpd=gpd.read_file("data/output/1_population_density/grid_pop_"+kdprov+"_1km_w_bu.gpkg"
                      ).query('(pop>0) or (p_builtup>0)').reset_index()

In [4]:
data_gpd['Name']=data_gpd.index.astype(str).str.zfill(7)

In [5]:
##Check cluster centre
data_cl_centre=data_gpd.query('pop>=1500')

In [6]:
from pysal.lib import weights

In [7]:
wr = weights.contiguity.Rook.from_dataframe(data_cl_centre)
data_cl_centre['cluster_label']=wr.component_labels

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 154 disconnected components.
 There are 98 islands with ids: 2, 3, 18, 19, 25, 31, 36, 47, 48, 49, 50, 52, 60, 67, 85, 89, 90, 91, 92, 93, 94, 95, 97, 99, 101, 109, 110, 115, 128, 144, 157, 191, 221, 253, 288, 290, 327, 376, 377, 378, 379, 402, 409, 435, 455, 456, 457, 482, 501, 502, 551, 565, 574, 576, 586, 594, 597, 598, 599, 617, 624, 625, 634, 641, 646, 655, 656, 662, 669, 682, 691, 697, 700, 703, 704, 706, 708, 710, 714, 715, 722, 725, 728, 735, 737, 739, 740, 741, 748, 754, 755, 760, 765, 766, 767, 773, 774, 775.
  warnings.warn(message)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [8]:
import pandas as pd

In [9]:
tab_pop_u_centre_cand=pd.pivot_table(data_cl_centre,
                                     index='cluster_label',values='pop',aggfunc='sum')

In [10]:
list_u_centre=tab_pop_u_centre_cand.query('pop>50000').reset_index().cluster_label.tolist()

In [11]:
u_centre=data_cl_centre.query('cluster_label in @list_u_centre')

In [12]:
data_gpd['is_U_CENTRE']=data_gpd.Name.apply(lambda y: 1 if y in u_centre.Name.tolist() else 0)

In [13]:
#Filling gap of urbancentre => check 8 contiguity (queen)
init_u_centre=data_gpd.is_U_CENTRE.sum()
u_centre_up=0
print('N GRID WITH URBAN CENTRE:',init_u_centre)
while u_centre_up!=init_u_centre:
    init_u_centre=data_gpd.is_U_CENTRE.sum()
    wr = weights.contiguity.Queen.from_dataframe(data_gpd)
    data_gpd['neighbors']=wr.neighbors.values()
    temp=data_gpd.copy()
    for index, row in data_gpd.iterrows():
        if row['is_U_CENTRE']!=1:
            y_urbancentre=0
            if len(row['neighbors'])>4:
                for y in row['neighbors']:
                    if temp.loc[y,'is_U_CENTRE']==1:
                        y_urbancentre+=1
                if y_urbancentre>4:
                    data_gpd.at[index,'is_U_CENTRE']=1
    u_centre_up=data_gpd.is_U_CENTRE.sum()
    print('=====================')
    print('N GRID AFTER GAP FILLING:', u_centre_up)
print('==================')
u_centre=data_gpd.is_U_CENTRE.sum()
print('N GRID COMPLETED AFTER GAP FILLING',u_centre)

N GRID WITH URBAN CENTRE: 497


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7 disconnected components.
 There is 1 island with id: 115.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 560


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7 disconnected components.
 There is 1 island with id: 115.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 585


...

N GRID AFTER GAP FILLING: 611


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7 disconnected components.
 There is 1 island with id: 115.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 612


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7 disconnected components.
 There is 1 island with id: 115.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 612
N GRID COMPLETED AFTER GAP FILLING 612


In [14]:
data_gpd['is_U_CENTRE_w_BU']=data_gpd.apply(lambda y:1 if (y.p_builtup>0.5)or(y.is_U_CENTRE==1) else 0,axis=1)

In [15]:
data_gpd=data_gpd[['Name','pop','geometry','p_builtup','is_U_CENTRE_w_BU']]

In [16]:
data_gpd

,Name,pop,geometry,p_builtup,is_U_CENTRE_w_BU
0,0000000,0,"POLYGON ((11397600.000 -1008400.000, 11397600....",0.059174,0
1,0000001,0,"POLYGON ((11397600.000 -1009400.000, 11397600....",0.307061,0
2,0000002,0,"POLYGON ((11397600.000 -1010400.000, 11397600....",0.157641,0
3,0000003,0,"POLYGON ((11397600.000 -1011400.000, 11397600....",0.009950,0
4,0000004,0,"POLYGON ((11397600.000 -1012400.000, 11397600....",0.002261,0
...,...,...,...,...,...
5051,0005051,0,"POLYGON ((11520600.000 -1034400.000, 11520600....",0.033995,0
5052,0005052,0,"POLYGON ((11520600.000 -1035400.000, 11520600....",0.024955,0
5053,0005053,0,"POLYGON ((11520600.000 -1036400.000, 11520600....",0.033803,0
5054,0005054,0,"POLYGON ((11520600.000 -1037400.000, 11519600....",0.012341,0


In [17]:
data_u_cluster=data_gpd.query('is_U_CENTRE_w_BU!=1').query('pop>=300')

In [18]:
wr = weights.contiguity.Queen.from_dataframe(data_u_cluster)
data_u_cluster['cluster_label']=wr.component_labels

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 64 disconnected components.
 There are 30 islands with ids: 0, 310, 344, 351, 389, 401, 412, 422, 436, 565, 673, 674, 784, 844, 958, 1125, 1166, 1276, 1306, 1439, 1706, 1819, 2053, 2088, 2101, 2125, 2164, 2180, 2376, 2398.
  warnings.warn(message)


In [19]:
tab_pop_u_cluster_cand=pd.pivot_table(data_u_cluster,
                                     index='cluster_label',values='pop',aggfunc='sum')
list_u_cluster=tab_pop_u_cluster_cand.query('pop>=5000').reset_index().cluster_label.tolist()

In [20]:
u_cluster=data_u_cluster.query('cluster_label in @list_u_cluster')

In [21]:
data_gpd['is_U_CLUSTER']=data_gpd.Name.apply(lambda y: 1 if y in u_cluster.Name.tolist() else 0)

In [22]:
data_gpd['classification']=data_gpd.apply(lambda y: 'CENTRE' 
                                          if y.is_U_CENTRE_w_BU==1 else ('CLUSTER' if y.is_U_CLUSTER==1 else 'RURAL'),axis=1)

In [23]:
data_gpd.classification.unique()

array(['RURAL', 'CLUSTER', 'CENTRE'], dtype=object)

In [24]:
data_gpd.to_file('data/output/2_grid_degurba/degurba_'+kdprov+'00.gpkg',driver='GPKG')